In [1]:
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Flatten, Activation, RepeatVector, Permute, Lambda, \
 Bidirectional, TimeDistributed, Dropout, Conv1D, GlobalMaxPool1D, multiply, concatenate
from tensorflow.keras.preprocessing.sequence  import  pad_sequences
from tensorflow.keras.layers import Layer
from itertools import product
import keras.backend as K

c:\Users\ayush\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tx_df = pd.read_csv("train_dataframe.csv")
tx_df  = tx_df[["text_id", "title", "text"]]
tx_df.columns = ["id1", "title1", "text1"]

C:\Users\ayush\AppData\Local\Temp\ipykernel_23920\154015079.py:1: DtypeWarning: Columns (21,33,34,35,36,37,38,41,46,48,49,60,61,62,66,67,69,71,72,73,74,75,76,77,78,79,82,83,84,85,86,87,88,89,90,91,92,94,95,96,97,98,100,101,102,105,106,107,108,110,111,112,113,114,116,117,118,119,122,123,124,125,126,127,128,129,131,132,133,134,135,137,138,139,140,141,142,143,144,146,147,148,149,150,151,153,154,155,156,159,160,161,162,163,164,165,169,171,173,174,175,176,177,178,179,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,238,239,240,241,242,243,244,245,247,248,249,250,251,252,253,254,255,256,257,258,259,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,278,279,280,281,282,283,285,286,287,289,290,291,292,293,294,295,296,297,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,3

In [3]:
pair = tx_df.iloc[1::2,:]
tx_df.drop(index=pair.index.tolist(), axis=0, inplace=True)
tx_df["id2"] = pair.id1.tolist()
tx_df["title2"] = pair.title1.tolist()
tx_df["text2"] = pair.text1.tolist()

In [4]:
tx_df

,id1,title1,text1,id2,title2,text2
0,1484084337,Virginia man arrested in fatal DUI crash in We...,"MARTINSBURG, W.Va. — A suspected drunken drive...",1484110209,Haiti’s leader marks independence day amid sec...,"PORT-AU-PRINCE, Haiti — Haitian President Jove..."
2,1484396422,Guyana: Three injured after car crashes into u...,(NEWS ROOM GUYANA) — Three persons are current...,1483924666,"Apes, monkeys among more than 30 animals kille...",BERLIN - A fire raced through a zoo in western...
4,1484698254,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,(Breitbart) – President Donald Trump welcomed ...,1483758694,"Trump says he does not expect war with Iran, ‘...","PALM BEACH, United States — US President Donal..."
6,1576314516,Zomato Buys Uber's Food Delivery Business in I...,Uber has sold its online food-ordering busines...,1576455088,Indian Online Food Delivery Market to Hit $8 B...,Rapid digitisation and growth in both online b...
8,1484036253,"India approves third moon mission, months afte...",BENGALURU (Reuters) - India has approved its t...,1483894099,NaN,NaN
...,...,...,...,...,...,...
7714,1486926924,21st Austria weekly - Foreign trade statistics...,Regional foreign trade: In the first half year...,1486926924,21st Austria weekly - Foreign trade statistics...,Regional foreign trade: In the first half year...
7716,1521779533,"Berlin: Ein Toter, vier Verletzte bei Schießer...",Update 11:20: Polizei korrigiert Zahl der Verl...,1484662752,Gunman Opens Fire At Cafe In Berlin City Centr...,The shots were likely related to a robbery att...
7718,1537102350,CDU-Vorsitz: Ein Chef auf Probe,Wer beim Sonderparteitag im April zum neuen Vo...,1531416451,BERLIN – Laschet and Merz announce CDU candidacy,North Rhine-Westphalia’s Minister-President Ar...
7720,1593108703,Neu-Isenburg: Heinz Schickedanz ist im Alter v...,Trauer um ein Isenburger Urgestein - Als „Blät...,1546957384,SMa.r.t. Column: We Must Act Now,"Like many others in town, our group consists o..."


In [5]:
url_df = pd.read_csv("train.csv")
url_df[['id1', 'id2']] = url_df["pair_id"].apply(lambda x: pd.Series(str(x).split("_")))
url_df.drop("pair_id", axis=1, inplace=True)
url_df.drop(url_df[url_df.duplicated(['id1', 'id2'])].index, axis=0, inplace=True)
overalls = []
remove = []
for id in tx_df["id1"].tolist():
    try:
        overalls.append(url_df[url_df["id1"] == str(id)].Overall.values[0])
    except:
        overalls.append(url_df[url_df["id2"] == str(id)].Overall.values[0])
        
tx_df["overall"] = overalls

In [6]:
tx_df

,id1,title1,text1,id2,title2,text2,overall
0,1484084337,Virginia man arrested in fatal DUI crash in We...,"MARTINSBURG, W.Va. — A suspected drunken drive...",1484110209,Haiti’s leader marks independence day amid sec...,"PORT-AU-PRINCE, Haiti — Haitian President Jove...",4.000000
2,1484396422,Guyana: Three injured after car crashes into u...,(NEWS ROOM GUYANA) — Three persons are current...,1483924666,"Apes, monkeys among more than 30 animals kille...",BERLIN - A fire raced through a zoo in western...,3.666667
4,1484698254,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,(Breitbart) – President Donald Trump welcomed ...,1483758694,"Trump says he does not expect war with Iran, ‘...","PALM BEACH, United States — US President Donal...",2.333333
6,1576314516,Zomato Buys Uber's Food Delivery Business in I...,Uber has sold its online food-ordering busines...,1576455088,Indian Online Food Delivery Market to Hit $8 B...,Rapid digitisation and growth in both online b...,2.000000
8,1484036253,"India approves third moon mission, months afte...",BENGALURU (Reuters) - India has approved its t...,1483894099,NaN,NaN,1.250000
...,...,...,...,...,...,...,...
7714,1486926924,21st Austria weekly - Foreign trade statistics...,Regional foreign trade: In the first half year...,1486926924,21st Austria weekly - Foreign trade statistics...,Regional foreign trade: In the first half year...,4.000000
7716,1521779533,"Berlin: Ein Toter, vier Verletzte bei Schießer...",Update 11:20: Polizei korrigiert Zahl der Verl...,1484662752,Gunman Opens Fire At Cafe In Berlin City Centr...,The shots were likely related to a robbery att...,4.000000
7718,1537102350,CDU-Vorsitz: Ein Chef auf Probe,Wer beim Sonderparteitag im April zum neuen Vo...,1531416451,BERLIN – Laschet and Merz announce CDU candidacy,North Rhine-Westphalia’s Minister-President Ar...,2.000000
7720,1593108703,Neu-Isenburg: Heinz Schickedanz ist im Alter v...,Trauer um ein Isenburger Urgestein - Als „Blät...,1546957384,SMa.r.t. Column: We Must Act Now,"Like many others in town, our group consists o...",4.000000


In [7]:
pd.set_option("display.max_rows", 200)

In [8]:
tx_df.dropna(axis=0, how="any", inplace=True)

In [9]:
#distribution of the languages used in the corpus
url_df["url1_lang"].value_counts()

en    1800
de    1388
es     570
tr     465
pl     349
ar     274
fr      72
Name: url1_lang, dtype: int64

In [10]:
def get_seq_lens(tx_df: pd.DataFrame, col: list[str]) -> list[list[int]]:
    col_seq_len = []
    text2_seq_len = []
    title1_seq_len = []
    title2_seq_len = []
    df = tx_df.copy()
    for i in tqdm(tx_df.index):
        df.loc[i,col] = df.loc[i,col].apply(lambda x: len(x)).copy()
    for c in col:
        col_seq_len.append(df[c].tolist())
    return col_seq_len

In [11]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
enc = tokenizer.encode("Hello there!")
dec = tokenizer.decode(enc)
print("Encode: " + str(enc))
print("Decode: " + str(dec))
print("[CLS]: " + str(enc[0]))
print("[SEP]: " + str(enc[4]))
print("[PAD]: " + str(tokenizer.encode("[PAD]")[1]))

print("="*250)

print("Encoding the data in the dataframe")
for i in tqdm(tx_df.index):
    tx_df.loc[i,["text1", "text2", "title1", "title2"]] = tx_df.loc[i,["text1", "text2", "title1", "title2"]].apply(tokenizer.encode).copy()

print("="*250)
    
print("Getting the corpus source length")    
#text1_seq_len, text2_seq_len, title1_seq_len, title2_seq_len = get_seq_lens(tx_df)

Encode: [101, 29155, 10768, 106, 102]
Decode: [CLS] hello there! [SEP]
[CLS]: 101
[SEP]: 102
[PAD]: 0
Encoding the data in the dataframe


100%|██████████| 3682/3682 [01:16<00:00, 48.26it/s]

Getting the corpus source length


In [12]:
def  split_and_zero_padding(df, max_seq_length): # Adjust tokens length
 
 # Convert the training set matrix into a dictionary
#  X  = {'left' : df['text1'], 'right':df['text2']}
 for i in tqdm(df.index):
   df.at[i,'text1'] =  list(pad_sequences(df.loc[i, ['text1']].tolist(), padding='post', maxlen=max(max_seq_length[0]), value=tokenizer.encode("[PAD]")[1])[0])
   df.at[i,'text2'] =  list(pad_sequences(df.loc[i, ['text2']].tolist(), padding='post', maxlen=max(max_seq_length[1]), value=tokenizer.encode("[PAD]")[1])[0])
 # Adjust to the specified length
#  for  dataset, side in product([X], ['left', 'right']):
#     dataset[side] =  pad_sequences (dataset[side], padding='pre', truncating='post', maxlen=max_seq_length[0])
 
 return df
 
 
class ManDist(Layer): # Manhattan distance calculation encapsulated into keras layer
 
 # Initialize the ManDist layer, no parameter input is required at this time
 def __init__(self, ** kwargs):
    self.result = None
    super(ManDist, self).__init__(** kwargs)
 
 # Automatically create the ManDist layer
 def build(self, input_shape):
    super(ManDist, self).build(input_shape)
 
 # Compute the Manhattan distance
 def call(self, x, ** kwargs):
    self.result = K.exp(-K.sum(K.abs(x[0] -  x[1]), axis=1, keepdims=True))
    return  self.result
 
 # return result
 def compute_output_shape(self, input_shape):
    return K.int_shape(self.result)

In [15]:
X_train = X_train.apply(tf.convert_to_tensor)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [14]:
# Split the training set
x = tx_df[['text1', 'text2']]
y = tx_df['overall'].astype("int32")
X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size=0.1)
tr_seq_len_tx1, tr_seq_len_tx2 = get_seq_lens(X_train, ['text1', 'text2'])
X_train = split_and_zero_padding(X_train, [tr_seq_len_tx1, tr_seq_len_tx2])
val_seq_len_tx1, val_seq_len_tx2 = get_seq_lens(X_val, ['text1', 'text2'])
X_val = split_and_zero_padding(X_val, [val_seq_len_tx1, val_seq_len_tx2])
 
# Convert labels to numbers
Y_train = Y_train.values
Y_val = Y_val.values

X_train = X_train.tolist().apply(tf.convert_to_tensor)
X_val = X_val.tolist().apply(tf.convert_to_tensor)
Y_train = tf.convert_to_tensor(Y_train)
Y_val = tf.convert_to_tensor(Y_val)

# Confirm that the data is ready and correct
assert X_train['text1'].shape == X_train['text2'].shape
assert len(X_train['text1']) == len(Y_train)

100%|██████████| 369/369 [22:10<00:00,  3.61s/it] 


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [29]:
X_train

,text1,text2
5652,"[101, 18552, 168, 28441, 17118, 20770, 10433, ...","[101, 51356, 37443, 62738, 21888, 23846, 14337..."
3956,"[101, 35032, 93162, 51915, 13856, 23571, 165, ...","[101, 165, 32917, 23518, 29381, 32309, 11337, ..."
2342,"[101, 37783, 10372, 13524, 10114, 25839, 11310...","[101, 25650, 117, 10377, 10806, 117, 20657, 10..."
2616,"[101, 122, 10108, 10148, 11530, 11848, 15535, ...","[101, 10151, 16061, 11636, 11416, 10246, 17397..."
7016,"[101, 12976, 58817, 10111, 34827, 117, 10121, ...","[101, 10144, 53693, 10592, 10144, 53693, 10592..."
...,...,...
5216,"[101, 18149, 10102, 58177, 131, 10494, 50967, ...","[101, 10109, 11707, 117, 10265, 36447, 10134, ..."
4568,"[101, 107, 11312, 14650, 17134, 80159, 11423, ...","[101, 48135, 11008, 68612, 46831, 10114, 58159..."
6244,"[101, 10109, 10608, 10102, 14451, 15865, 16113...","[101, 11357, 10106, 11831, 24755, 10102, 154, ..."
3884,"[101, 29104, 11266, 19706, 10732, 25473, 18908...","[101, 14072, 90652, 10402, 12380, 10136, 57950..."


In [31]:
X_train

,text1,text2
5652,"[101, 18552, 168, 28441, 17118, 20770, 10433, ...","[101, 51356, 37443, 62738, 21888, 23846, 14337..."
3956,"[101, 35032, 93162, 51915, 13856, 23571, 165, ...","[101, 165, 32917, 23518, 29381, 32309, 11337, ..."
2342,"[101, 37783, 10372, 13524, 10114, 25839, 11310...","[101, 25650, 117, 10377, 10806, 117, 20657, 10..."
2616,"[101, 122, 10108, 10148, 11530, 11848, 15535, ...","[101, 10151, 16061, 11636, 11416, 10246, 17397..."
7016,"[101, 12976, 58817, 10111, 34827, 117, 10121, ...","[101, 10144, 53693, 10592, 10144, 53693, 10592..."
...,...,...
5216,"[101, 18149, 10102, 58177, 131, 10494, 50967, ...","[101, 10109, 11707, 117, 10265, 36447, 10134, ..."
4568,"[101, 107, 11312, 14650, 17134, 80159, 11423, ...","[101, 48135, 11008, 68612, 46831, 10114, 58159..."
6244,"[101, 10109, 10608, 10102, 14451, 15865, 16113...","[101, 11357, 10106, 11831, 24755, 10102, 154, ..."
3884,"[101, 29104, 11266, 19706, 10732, 25473, 18908...","[101, 14072, 90652, 10402, 12380, 10136, 57950..."


In [33]:
for i in X_train.index:
    X_train.loc[i, "text1"] = tf.convert_to_tensor(X_train.loc[i, "text1"])
    X_train.loc[i, "text2"] = tf.convert_to_tensor(X_train.loc[i, "text2"])
    
for i in X_val.index:
    X_val.loc[i, "text1"] = tf.convert_to_tensor(X_val.loc[i, "text1"])
    X_val.loc[i, "text2"] = tf.convert_to_tensor(X_val.loc[i, "text2"])

In [30]:
X_train = X_train.apply(lambda x: tf.convert_to_tensor(list(x)))

ValueError: If using all scalar values, you must pass an index

In [ ]:
X_val = X_val.apply(lambda x: tf.convert_to_tensor(list(x)))

In [27]:
Y_train = tf.convert_to_tensor(Y_train)
Y_val = tf.convert_to_tensor(Y_val)

In [22]:
X_train = X_train.apply(tf.convert_to_tensor, axis=1)
X_val = X_val.apply(tf.convert_to_tensor, axis=1)

Y_train = tf.convert_to_tensor(Y_train)
Y_val = tf.convert_to_tensor(Y_val)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [23]:
X_train

,text1,text2
5652,"[101, 18552, 168, 28441, 17118, 20770, 10433, ...","[101, 51356, 37443, 62738, 21888, 23846, 14337..."
3956,"[101, 35032, 93162, 51915, 13856, 23571, 165, ...","[101, 165, 32917, 23518, 29381, 32309, 11337, ..."
2342,"[101, 37783, 10372, 13524, 10114, 25839, 11310...","[101, 25650, 117, 10377, 10806, 117, 20657, 10..."
2616,"[101, 122, 10108, 10148, 11530, 11848, 15535, ...","[101, 10151, 16061, 11636, 11416, 10246, 17397..."
7016,"[101, 12976, 58817, 10111, 34827, 117, 10121, ...","[101, 10144, 53693, 10592, 10144, 53693, 10592..."
...,...,...
5216,"[101, 18149, 10102, 58177, 131, 10494, 50967, ...","[101, 10109, 11707, 117, 10265, 36447, 10134, ..."
4568,"[101, 107, 11312, 14650, 17134, 80159, 11423, ...","[101, 48135, 11008, 68612, 46831, 10114, 58159..."
6244,"[101, 10109, 10608, 10102, 14451, 15865, 16113...","[101, 11357, 10106, 11831, 24755, 10102, 154, ..."
3884,"[101, 29104, 11266, 19706, 10732, 25473, 18908...","[101, 14072, 90652, 10402, 12380, 10136, 57950..."


In [36]:
from tensorflow.keras.initializers import GlorotUniform

In [38]:
def shared_model(_input, max_seq_length):
 # word vectorization
 embedded = Embedding(len(tokenizer.get_vocab()), 128, input_shape=(max_seq_length,), embeddings_initializer=GlorotUniform(seed=0))(_input)
 
 # Multilayer Bi-LSTM
 activations = Bidirectional(LSTM(n_hidden, return_sequences=True), merge_mode='concat')(embedded)
 activations = Bidirectional(LSTM(n_hidden, return_sequences=True), merge_mode='concat')(activations)
 
 # dropout
 # activations = Dropout(0.5)(activations)
 
 # Attention
 attention = TimeDistributed(Dense(1, activation='tanh'))(activations)
 attention = Flatten()(attention)
 attention = Activation('softmax')(attention)
 attention = RepeatVector(n_hidden * 2)(attention)
 attention = Permute([2, 1])(attention)
 sent_representation = multiply([activations, attention])
 sent_representation = Lambda(lambda xin: K.sum(xin, axis=1))(sent_representation)
 
 # dropout
 # sent_representation = Dropout(0.1)(sent_representation)
 
 return sent_representation
 
 
# def shared_model_cnn(_input):
#  # word vectorization
#  embedded = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_shape=(max_seq_length,),
#  trainable=False)(_input)
 
#  # CNN
#  activations = Conv1D(250, kernel_size=5, activation='relu')(embedded)
#  activations = GlobalMaxPool1D()(activations)
#  activations = Dense(250, activation='relu')(activations)
#  activations = Dropout(0.3)(activations)
#  activations = Dense(1, activation='sigmoid')(activations)
 
#  return activations
 
 
# ----------------- Main function ----------------- #
 
if __name__ == '__main__':
 
 # hyperparameter
 batch_size = 1024
 n_epoch  =  9
 n_hidden = 50
 
 left_input = Input(shape=(max(tr_seq_len_tx1),), dtype='float32')
 right_input = Input(shape=(max(tr_seq_len_tx2),), dtype='float32')
 left_sen_representation = shared_model(left_input, max(tr_seq_len_tx1))
 right_sen_representation = shared_model(right_input, max(tr_seq_len_tx2))
 
 # Introduce the Manhattan distance, and then pass the original vector on the transformed concat through a multi-layer DNN to make a non-linear transformation and sigmoid similarity
 # Did not use the Mahalanobis distance mentioned in https://zhuanlan.zhihu.com/p/31638132, tried the Manhattan distance, dot product and cos, and the Manhattan effect is the best
 man_distance = ManDist()([left_sen_representation, right_sen_representation])
 sen_representation = concatenate([left_sen_representation, right_sen_representation, man_distance])
 similarity = Dense(1, activation='sigmoid')(Dense(2)(Dense(4)(Dense(16)(sen_representation))))
 model = Model(inputs=[left_input, right_input], outputs=[similarity])
 
 model.compile(loss='mean_squared_error', optimizer="Adam", metrics=['accuracy'])
 model.summary()
 

 malstm_trained = model.fit([tf.convert_to_tensor(X_train['text1'].tolist()), tf.convert_to_tensor(X_train['text2'].tolist())], Y_train,
 batch_size=batch_size, epochs=2,
 validation_data=([tf.convert_to_tensor(X_val['text1'].tolist()), tf.convert_to_tensor(X_train['text2'].tolist())], Y_val))
 
 # Plot accuracy
 import  matplotlib
 matplotlib.use('Agg')
 import  matplotlib . pyplot  as  plt
 
 plt.subplot(211)
 plt.plot(malstm_trained.history['acc'])
 plt.plot(malstm_trained.history['val_acc'])
 plt.title('Model Accuracy')
 plt.ylabel('Accuracy')
 plt . xlabel ( 'Epoch' )
 plt.legend(['Train', 'Validation'], loc='upper left')
 
 # Plot loss
 plt.subplot(212)
 plt.plot(malstm_trained.history['loss'])
 plt.plot(malstm_trained.history['val_loss'])
 plt.title('Model Loss')
 plt.ylabel('Loss')
 plt . xlabel ( 'Epoch' )
 plt.legend(['Train', 'Validation'], loc='upper right')
 
 plt.tight_layout(h_pad=1.0)
 plt.savefig('./data/history-graph.png')
 
#  model.save(savepath)
#  print(str(malstm_trained.history['val_acc'][-1])[:6] +
#  "(max: " + str(max(malstm_trained.history['val_acc']))[:6] + ")")
#  print("Done.")

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 51471)]      0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 60046)]      0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 51471, 128)   13552512    ['input_7[0][0]']                
                                                                                                  
 embedding_5 (Embedding)        (None, 60046, 128)   13552512    ['input_8[0][0]']                
                                                                                            

ResourceExhaustedError: Graph execution error:

Detected at node 'model_2/embedding_4/embedding_lookup' defined at (most recent call last):
    File "c:\Users\ayush\miniconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\ayush\miniconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\ayush\miniconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\ayush\miniconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\ayush\miniconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\ayush\miniconda3\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\ayush\miniconda3\lib\asyncio\base_events.py", line 601, in run_forever
  